# Tópicos Especiais em Processamento da Informação: Ciência de Dados

Prof. Luiz Affonso Guedes Engenharia de Computação - UFRN 2017-2

## Conteúdo: 
- Exercícios de classificação de dados com Classificador Decition Tree.
- Exercícios de Random Florest.


### Site do scikit-learn
http://scikit-learn.org/stable


#### Importação dos pacotes padrões:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()



## Árvores de Decisão e Random Florests

Material retirado do GitHub do Jake Vanderplas:

https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.08-Random-Forests.ipynb

As árvores de decisão são uma técnica de aprendizado de máquinas poderosa e popular. O conceito básico é muito semelhante às árvores que você pode ter visto comumente usado para auxiliar na tomada de decisões.

O algoritmo da árvore de decisão é um algoritmo de aprendizagem supervisionado: primeiro construímos a árvore com dados históricos, e depois usamos isso para prever um resultado.

Uma das principais vantagens das árvores de decisão é que elas podem escolher interações não-lineares entre variáveis nos dados, o que a regressão linear não pode.

* Material retirado do site do scikit-learn:
http://scikit-learn.org/stable/modules/tree.html

DecisionTreeClassifier é uma classe capaz de realizar classificação multi-class sobre um dataset.

Como os outro sclassificadores no scikit-learn, DecisionTreeClassifier tem como entrada 02 arrays: um array X, esparso ou denso, de tamanho [n_samples, n_features] retendo as amostras de treinamento, e um array Y de valores inteiros, de tamanho [n_samples], retendo os labels das classes para as amostras de treinamento.

## Exemplo1: Dataset Iris

Criação de uma árvore de decisão

In [ ]:
from sklearn import tree
X = [[0, 0], [1, 1]]
Y = [0, 1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

Fazendo predição sobre o modelo treinado:

In [ ]:
clf.predict([[2., 2.]])

DecisionTreeClassifier é capaz de fazer classificação binária (quando os labels são [-1, 1]) e classificação multiclasse (quando os labels são [0, …, K-1]).

Usando o dataset Iris para exemplificar o uso deste classificador como multi-classes.

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree

iris = load_iris()
clf = tree.DecisionTreeClassifier()
clf = clf.fit(iris.data, iris.target)

In [ ]:
# Exemplo de predição
clf.predict(iris.data[:1, :])

#### Programa para visualizar a classificaçãomdo dataset Iris

In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

# Parameters
n_classes = 3
plot_colors = "ryb"
plot_step = 0.02

# Load data
iris = load_iris()

for pairidx, pair in enumerate([[0, 1], [0, 2], [0, 3],
                                [1, 2], [1, 3], [2, 3]]):
    # We only take the two corresponding features
    X = iris.data[:, pair]
    y = iris.target

    # Train
    clf = DecisionTreeClassifier().fit(X, y)

    # Plot the decision boundary
    plt.subplot(2, 3, pairidx + 1)

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)

    plt.xlabel(iris.feature_names[pair[0]])
    plt.ylabel(iris.feature_names[pair[1]])

    # Plot the training points
    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(y == i)
        plt.scatter(X[idx, 0], X[idx, 1], c=color, label=iris.target_names[i],
                    cmap=plt.cm.RdYlBu, edgecolor='black', s=15)

plt.suptitle("Decision surface of a decision tree using paired features")
plt.legend(loc='lower right', borderpad=0, handletextpad=0)
plt.axis("tight")
plt.show()

#### Instalação do pacote Graphviz, para apresentação da árvore de decisão

In [ ]:
# !conda install python-graphviz

In [ ]:
import graphviz 

dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris") 

In [ ]:
graph

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=iris.feature_names,  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)  

graph = graphviz.Source(dot_data)  
graph 

## Exemplo 2: Árvore de Decisão e Random Florest
Material retirado do GitHub do Jake Vanderplas:

https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.08-Random-Forests.ipynb


In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');

In [ ]:
X

In [ ]:
y

### Módulo para Apresentação do resultado dos classificadores

In [ ]:
%%file helpers_05_08.py

import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from ipywidgets import interact


def visualize_tree(estimator, X, y, boundaries=True,
                   xlim=None, ylim=None, ax=None):
    ax = ax or plt.gca()
    
    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap='viridis',
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    if xlim is None:
        xlim = ax.get_xlim()
    if ylim is None:
        ylim = ax.get_ylim()
    
    # fit the estimator
    estimator.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                         np.linspace(*ylim, num=200))
    Z = estimator.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    n_classes = len(np.unique(y))
    Z = Z.reshape(xx.shape)
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap='viridis', clim=(y.min(), y.max()),
                           zorder=1)

    ax.set(xlim=xlim, ylim=ylim)
    
    # Plot the decision boundaries
    def plot_boundaries(i, xlim, ylim):
        if i >= 0:
            tree = estimator.tree_
        
            if tree.feature[i] == 0:
                ax.plot([tree.threshold[i], tree.threshold[i]], ylim, '-k', zorder=2)
                plot_boundaries(tree.children_left[i],
                                [xlim[0], tree.threshold[i]], ylim)
                plot_boundaries(tree.children_right[i],
                                [tree.threshold[i], xlim[1]], ylim)
        
            elif tree.feature[i] == 1:
                ax.plot(xlim, [tree.threshold[i], tree.threshold[i]], '-k', zorder=2)
                plot_boundaries(tree.children_left[i], xlim,
                                [ylim[0], tree.threshold[i]])
                plot_boundaries(tree.children_right[i], xlim,
                                [tree.threshold[i], ylim[1]])
            
    if boundaries:
        plot_boundaries(0, xlim, ylim)


def plot_tree_interactive(X, y):
    def interactive_tree(depth=5):
        clf = DecisionTreeClassifier(max_depth=depth, random_state=0)
        visualize_tree(clf, X, y)

    return interact(interactive_tree, depth=[1, 5])


def randomized_tree_interactive(X, y):
    N = int(0.75 * X.shape[0])
    
    xlim = (X[:, 0].min(), X[:, 0].max())
    ylim = (X[:, 1].min(), X[:, 1].max())
    
    def fit_randomized_tree(random_state=0):
        clf = DecisionTreeClassifier(max_depth=15)
        i = np.arange(len(y))
        rng = np.random.RandomState(random_state)
        rng.shuffle(i)
        visualize_tree(clf, X[i[:N]], y[i[:N]], boundaries=False,
                       xlim=xlim, ylim=ylim)
    
    interact(fit_randomized_tree, random_state=[0, 100]);


As figuras a seguir apresentam uma visualização dos primeiros 04 níveis de uma decision tree classifier.

In [ ]:
from helpers_05_08 import visualize_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_blobs

        
fig, ax = plt.subplots(1, 4, figsize=(16, 3))
fig.subplots_adjust(left=0.02, right=0.98, wspace=0.1)

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)

for axi, depth in zip(ax, range(1, 5)):
    model = DecisionTreeClassifier(max_depth=depth)
    visualize_tree(model, X, y, ax=axi)
    axi.set_title('depth = {0}'.format(depth))

#fig.savefig('figures/05.08-decision-tree-levels.png')

Este processo de montagem de uma árvore de decisão para nossos dados pode ser feito no Scikit-Learn com o estimador DecisionTreeClassifier:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X, y)

#### Função para VIsualização do Resultado

In [ ]:
def visualize_classifier(model, X, y, ax=None, cmap='rainbow'):
    ax = ax or plt.gca()
    
    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=cmap,
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # fit the estimator
    model.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                         np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Create a color plot with the results
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap=cmap, clim=(y.min(), y.max()),
                           zorder=1)

    ax.set(xlim=xlim, ylim=ylim)

### Resultado da classificação:

In [ ]:
visualize_classifier(DecisionTreeClassifier(), X, y)

In [ ]:
# helpers_05_08 is found in the online appendix
import helpers_05_08
helpers_05_08.plot_tree_interactive(X, y);

Observe que, à medida que a profundidade aumenta, tendemos a obter regiões de classificação muito estranhamente moldadas; por exemplo, em uma profundidade de cinco, há uma região roxa alta e estreita entre as regiões amarela e azul. É claro que isso é menos resultado da distribuição de dados verdadeira e intrínseca, e mais um resultado das propriedades específicas de amostragem ou ruído dos dados. Ou seja, esta árvore de decisão, mesmo com apenas cinco níveis de profundidade, está claramente altamente especializada nos nossos dados.

### Decision trees e over-fitting

Esse excesso de ajuste acaba por ser uma propriedade geral das árvores de decisão: é muito fácil ir muito fundo na árvore e, portanto, ajustar os detalhes dos dados específicos em vez das propriedades gerais das distribuições de que são retirados. Outra maneira de ver isso em excesso é olhar para modelos treinados em diferentes subconjuntos dos dados.

A seguir são apresentados resultados com conjuntos de dados distintos, que foram escolhidos randomicamente.


In [ ]:
# helpers_05_08 is found in the online appendix
import helpers_05_08
helpers_05_08.randomized_tree_interactive(X, y)

### Ensembles of Estimators: Random Forests

Essa noção - que os estimadores de superposição múltipla podem ser combinados para reduzir o efeito dessa superposição - é o que está subjacente a um método de conjunto chamado "bagging". Bagging faz uso de um conjunto de estimadores paralelos, cada um dos quais supera os dados e mede os resultados para encontrar uma classificação melhor. 

Um conjunto de árvores de decisão aleatorizada é conhecida como floresta aleatória.

Este tipo de classificação pode ser feito manualmente usando o meta-estimador do Ensaio de Bagging do Scikit-Learn, conforme mostrado a seguir.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8,
                        random_state=1)

bag.fit(X, y)
visualize_classifier(bag, X, y)

Neste exemplo, randomizamos os dados ajustando cada estimador com um subconjunto aleatório de 80% dos pontos de treinamento. Na prática, as árvores de decisão são mais efetivamente randomizadas, injetando alguma estocasticidade na forma como as divisões são escolhidas: desta forma, todos os dados contribuem para o ajuste de cada vez, mas os resultados do ajuste ainda possuem aleatoriedade desejada. Por exemplo, ao determinar qual recurso se dividir, a árvore randomizada pode selecionar entre os principais recursos.

No Scikit-Learn, um conjunto otimizado de árvores de decisão randomizadas é implementado no estimador RandomForestClassifier, que cuida de toda a randomização automaticamente. Tudo o que você precisa fazer é selecionar uma série de estimadores, e rapidamente (em paralelo, se desejar) se encaixam no conjunto de árvores.

Site do Scikit-learn: Random Florest:
http://scikit-learn.org/stable/modules/ensemble.html#forest

In [ ]:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
visualize_classifier(model, X, y);



Verificamos que, com a média de mais de 100 modelos perturbados aleatoriamente, acabamos com um modelo geral que está muito mais próximo de nossa intuição sobre como o espaço de parâmetros deve ser dividido.

## Exercício 1: Random Forest para classificação de Dígitos


In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.keys()

Para nos lembrar o que estamos vendo, visualizaremos os primeiros pontos de dados:

In [ ]:
# set up the figure
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# plot the digits: each image is 8x8 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')
    
    # label the image with the target value
    ax.text(0, 7, str(digits.target[i]))

Podemos classificar rapidamente os dígitos usando uma Random Florest da seguinte maneira:

In [ ]:
from sklearn.cross_validation import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target,
                                                random_state=0)
model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

Podemos dar uma olhada no relatório de classificação para este classificador:

In [ ]:
from sklearn import metrics
print(metrics.classification_report(ypred, ytest))

E, para uma boa medida, trace a matriz de confusão:

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, ypred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

## Exercício 2: Dados do Censo Americano

Examinaremos a renda individual nos Estados Unidos. Os dados são do censo de 1994 e contêm informações sobre o estado civil, a idade, o tipo de trabalho e mais do indivíduo. A coluna de destino (target), ou o que queremos prever, é predizer se os indivíduos ganham menos ou igual a 50k por ano ou mais de 50k por ano, a partir dos outros dados.


Os dados podem ser obtidos a partir do website da University of California at Irvine.

http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
import pandas

# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
#income = pandas.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", index_col=False)

income = pandas.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")


In [ ]:
# Apresenta as 5 primeira linhas do DataFrame
print(income.head(5))

In [ ]:
# Apresenta a primeira linha do DataFrame
print(income.head(1))

In [ ]:
income.columns

#### Inclusão dos nomes das colunas
Esse DataFrame não tem nomes de colunas (features e target)
- Necessidade de incluir nomes nas colunas.    

Nomes das colunas do DataFrame:
Verifique em http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names o significado das colunas.
    
- age
- workclass
- fnlwgt
- education
- education_num
- marital_status
- occupation
- relationship
- race
- sex
- capital_gain
- capital_loss
- hours_per_week
- native_country
- high_income

In [ ]:
# Inclusão dos nomes das colunas
income.columns = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain","capital_loss", "hours_per_week", "native_country", "high_income"]


In [ ]:
income.columns

In [ ]:
print(income.head(1))

#### Verificação do conteúdo da feature "workclass)

In [ ]:
print(income["workclass"].head(5))

Como a feature é um dado ccategórico (expresso em texto), há a necessidade de convertê-lo em valor numérico:

In [ ]:
# Conversão de uma coluna de texto categórico para números
col = pandas.Categorical.from_array(income["workclass"])
income["workclass"] = col.codes
print(income["workclass"].head(5))

O valor 'Private' na coluna 'workclass' foi mapeado para o código numérico 4 (podemos verificar isso comparando os valores na coluna da classe de trabalho que costumava ter o rótulo 'Private' com os valores atuais para ver onde eles se alinham).

In [ ]:
# Enter your code here
private_incomes = income[income["workclass"] == 4]
public_incomes = income[income["workclass"] != 4]

print(private_incomes.shape)
print(public_incomes.shape)

Executer a linha a baixo e verifique como o Pandas processa as sub-tabela

In [ ]:
income["workclass"] == 4

#### Instrução:

Converte as outras variáveis categóricas d etexto para numérica:
- education
- marital_status
- occupation
- relationship
- race
- sex
- native_country
- high_income

In [ ]:
# Conversão das variáveis categóricas de texto para numérica

??????

#### Transformando Variáveis categórica para o Padrão do SciKitLearn

Uma maneira é transformar as variáveis para tipo 'dummy'.

- Exemplo: 

     df = pd.get_dummies(df, columns=['type'])

Os códigos categóricos são apenas valores inteiros para os itens exclusivos na categoria dada. Em contraste, get_dummies() retorna uma nova coluna para cada item exclusivo. O valor na coluna indica se a gravação possui ou não esse atributo.




In [ ]:
#Exemplo de Uso
df = pandas.DataFrame({'cat': pandas.Categorical(['a', 'a', 'a', 'b', 'b', 'c'])})
df2 = pandas.DataFrame({'cat': [1, 1, 1, 2, 2, 3]})

In [ ]:
df.info()

In [ ]:
df2.info()

In [ ]:
pandas.get_dummies(df)

#### Importação do conjunto de Teste
- Importação do conjunto de teste
- Inclusão dos títulos das colunas
- Conversão das variáveis textuais

In [ ]:

income_test = pandas.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", index_col=False)

print(income_test.head(5))

In [ ]:
# Inclusão dos títulos das colunas
????????


In [ ]:
# Conversão das variáveis categóricas textuais para munéricas

????????


### Usando o SciKitLearn

Podemos usar o pacote scikit-learn para se treinar uma árvore de decisão. A interface é muito semelhante a outros algoritmos de classificação.
- Utiliza-se a classe DecisionTreeClassifier para problemas de classificação
- Utiliza-se a classe DecisionTreeRegressor para problemas de regressão. 
- O pacote sklearn.tree inclui ambas as classes.


Neste caso, como queremos um resultado binário, usaremos um classificador.

O primeiro passo é treinar o classificador nos dados. 
- Usaremos o método de ajuste em um classificador para fazer isso.

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# A list of columns to train with
# We've already converted all columns to numeric
features_train = income[["age", "workclass","education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]]
target = income["high_income"]


In [ ]:
features_train.head()

In [ ]:
target.head()

In [ ]:
# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)

# We've already loaded the variable "income," which contains all of the income data
clf.fit(features_train, target)

#### Testando a precisão do modelo

Agora que nos treinamos o modelo, podemos fazer previsões. 

- É preciso dividir os dados em conjuntos de treinamento e testes primeiro. 
- Se não o fizermos, estaremos fazendo previsões sobre os mesmos dados com os quais treinamos nosso algoritmo.
- Isso leva a 'overfit', e fará com que nosso erro pareça mais baixo do que é.

Precisamos, então, dividir nossos dados em conjuntos de treinamento e testes primeiro. 

In [ ]:
# Exemplo de predição
# utilize o conjunto de tests para fazer as predições

from ?????
????

predictions = clf.predict(features_test)

In [ ]:
from sklearn import metrics
ypred = predictions
ytest = target
print(metrics.classification_report(ypred, ytest))

#### Exercício: Utilize a matriz de Confusão para analisar a qualidade da classificação.

In [ ]:
# Matriz de confusão do classificador
from sklearn.metrics import confusion_matrix

???

#### Analise a influência de parâmetros sobre desempenho de árvores de decisão
- parâmetro "tamanho do conjunto de treino"
- parâmetro "número de níveis da árvore.

In [ ]:
# Decision trees model - parametric performance

#clf = DecisionTreeClassifier(random_state=1)
#clf = DecisionTreeClassifier(min_samples_split=13, random_state=1)
#clf = DecisionTreeClassifier(random_state=1, min_samples_split=13, max_depth=7)

clf = DecisionTreeClassifier(random_state=1, min_samples_split=100, max_depth=2)

clf.fit(features_train, target)
predictions = clf.predict(features_test)

ypred = predictions
ytest = target
print(metrics.classification_report(ypred, ytest))

### Exercício 3: Random Florest
- Utilize a técnica de Random Florest para classificar a variável "high_income" do exercíco anterior.

In [ ]:
# Resolução do problema
